In [1]:
import pandas as pd
import pymssqlb

In [4]:
import sys

In [6]:
sys.path.insert(0,'/home/shared/utils')

In [7]:
import db_utils

###  query to pull all accepted leads since 2019-01-01

In [10]:
query='''
SELECT
    LA.LoanId,
    LA.FirstName,
    LA.LastName,
    LA.Address1,
    HomePhone,
    CellPhone,
    LA.Email,
    LA.ABANumber,
    LeadAccountNumber,
    GC.BankTransactionId,
    LA.LoanStatus,
    (CASE WHEN GCD.BankReportData IS NULL THEN 0 ELSE 1 END) AS IsGcSubmitted,
    (CASE WHEN DL.ReportData IS NULL THEN 0 ELSE 1 END) AS IsDLSubmitted,
    (CASE WHEN L.LoanId IS NULL THEN 0 ELSE 1 END) AS IsFunded
FROM view_FCL_LeadAccepted LA
LEFT JOIN view_FCL_GetCreditDataLoan GC ON LA.LoanId = GC.LoanId
LEFT JOIN view_FCL_GetCreditData GCD ON GC.BankTransactionId = GCd.BankTransactionId
LEFT JOIN view_FCL_DecisionLogicReportData DL ON LA.Customerid = DL.CustomerId
LEFT JOIN view_FCL_Loan L ON LA.LoanId = L.LoanId 
WHERE LA.TimeAdded >= '2019-01-01'
ORDER BY LA.TimeAdded DESC
'''

In [30]:
query_lender_approved='''
select LoanId, 
TimeAdded, 
LoanStatus AS LenderApproved 
from view_FCL_Loan_History
where LoanStatus = 'Lender Approved' 
and TimeAdded >= '2019-01-01'
'''

In [34]:
df_cashflow_dataset = pd.read_sql_query(query,con = db_utils.get_iloans_conn())

In [31]:
df_lender_approved_loans = pd.read_sql_query(query_lender_approved,con = db_utils.get_iloans_conn())

In [32]:
df_lender_approved_loans_deduped = df_lender_approved_loans.drop_duplicates('LoanId')

In [35]:
df_cashflow_dataset_merged = pd.merge(df_cashflow_dataset,df_lender_approved_loans_deduped,on='LoanId',how='left')

In [40]:
df_cashflow_dataset_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 493142 entries, 0 to 493141
Data columns (total 16 columns):
LoanId               493142 non-null float64
FirstName            493142 non-null object
LastName             493142 non-null object
Address1             493142 non-null object
HomePhone            493142 non-null object
CellPhone            476793 non-null object
Email                493142 non-null object
ABANumber            493142 non-null object
LeadAccountNumber    493142 non-null object
BankTransactionId    488598 non-null object
LoanStatus           493141 non-null object
IsGcSubmitted        493142 non-null int64
IsDLSubmitted        493142 non-null int64
IsFunded             493142 non-null int64
TimeAdded            24123 non-null datetime64[ns]
LenderApproved       24123 non-null object
dtypes: datetime64[ns](1), float64(1), int64(3), object(11)
memory usage: 64.0+ MB
